In [1]:
import pandas as pd
from tqdm import tqdm

import os

In [2]:
# 성능이 좋았던 모델부터 차례로 넣기 (파일 제목 앞을 점수로 넣어야 정렬이 제대로 된다.)
output_list= os.listdir('./output_csv')
output_list.sort(reverse=True)
output_list

['7059.csv', '6679.csv', '6593.csv', '6518.csv']

In [3]:
df_list = []

for output in output_list:
    df_list.append(pd.read_csv(f'./output_csv/{output}'))

In [4]:
submission = pd.DataFrame()
submission['image_id'] = df_list[0]['image_id']

In [5]:
PredictionString = []

for idx in tqdm(range(len(df_list[0]))):
    # 각 모델이 뽑은 pixel 넣을 리스트
    pixel_list = []
    
    for i in range(len(df_list)):
        pixel_list.append(df_list[i]['PredictionString'][idx].split(' '))

    result = ''

    for i in range(len(pixel_list[0])):
        pixel_count = {'0' : 0, '1' : 0, '2' : 0, 
                      '3' : 0, '4' : 0, '5' : 0,
                      '6' : 0, '7' : 0, '8' : 0,
                      '9' : 0, '10' : 0}
        
        # 각 모델이 뽑은 pixel count
        for j in range(len(pixel_list)):
            pixel_count[pixel_list[j][i]] += 1
        
        # 최댓값을 갖는 키값
        voted_pixel = [key for key, value in pixel_count.items() if value == max(pixel_count.values())]

        # voted_pixel이 1개인 경우
        if len(voted_pixel) == 1:
            result += voted_pixel[0] + ' '
        # voted_pixel이 2개 이상인 경우
        else:
            # 성능이 좋았던 모델부터 그 pixel이 voted_pixel에 있다면 result로 고르기
            for j in range(len(pixel_list)):
                pixel_candidate = pixel_list[j][i]

                if pixel_candidate in voted_pixel:
                    result += pixel_candidate + ' '
                    break
    
    # 마지막 공백 제거
    result = result[:-1]

    PredictionString.append(result)

100%|██████████| 624/624 [06:09<00:00,  1.69it/s]


In [6]:
submission['PredictionString'] = PredictionString
submission.to_csv('./hard_voted_output.csv', index=False)